In [13]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score

In [16]:
df = pd.read_csv('data/titanic_train.csv')

print(df.shape)
df.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Variable descriptions
- **pclass**: A proxy for socio-economic status (SES, i.e. upper, middle, lower): 1st/2nd/3rd
- **age**: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5
- **SibSp** (sibling/spouse)
- **Parch** (parent/child) The dataset defines family relations in this way
- **Cabin** has nan values, but 148 unique values like C85, C123 etc.
- **Ticket** has 681 unique values

### Peeking into the data

What is happening in columns like Ticket, Cabin etc.?

Ticket seems to have mostly unique values, but cabins have repeating values -- can convert this into a `cabin_types` column

In [17]:
print(df['Cabin'].unique())
print('High level types:', set([c[0] if type(c) == str else None for c in df['Cabin']]))

[nan 'C85' 'C123' 'E46' 'G6' 'C103' 'D56' 'A6' 'C23 C25 C27' 'B78' 'D33'
 'B30' 'C52' 'B28' 'C83' 'F33' 'F G73' 'E31' 'A5' 'D10 D12' 'D26' 'C110'
 'B58 B60' 'E101' 'F E69' 'D47' 'B86' 'F2' 'C2' 'E33' 'B19' 'A7' 'C49'
 'F4' 'A32' 'B4' 'B80' 'A31' 'D36' 'D15' 'C93' 'C78' 'D35' 'C87' 'B77'
 'E67' 'B94' 'C125' 'C99' 'C118' 'D7' 'A19' 'B49' 'D' 'C22 C26' 'C106'
 'C65' 'E36' 'C54' 'B57 B59 B63 B66' 'C7' 'E34' 'C32' 'B18' 'C124' 'C91'
 'E40' 'T' 'C128' 'D37' 'B35' 'E50' 'C82' 'B96 B98' 'E10' 'E44' 'A34'
 'C104' 'C111' 'C92' 'E38' 'D21' 'E12' 'E63' 'A14' 'B37' 'C30' 'D20' 'B79'
 'E25' 'D46' 'B73' 'C95' 'B38' 'B39' 'B22' 'C86' 'C70' 'A16' 'C101' 'C68'
 'A10' 'E68' 'B41' 'A20' 'D19' 'D50' 'D9' 'A23' 'B50' 'A26' 'D48' 'E58'
 'C126' 'B71' 'B51 B53 B55' 'D49' 'B5' 'B20' 'F G63' 'C62 C64' 'E24' 'C90'
 'C45' 'E8' 'B101' 'D45' 'C46' 'D30' 'E121' 'D11' 'E77' 'F38' 'B3' 'D6'
 'B82 B84' 'D17' 'A36' 'B102' 'B69' 'E49' 'C47' 'D28' 'E17' 'A24' 'C50'
 'B42' 'C148']
High level types: {'C', 'E', 'T', 'A', None

In [18]:
mean_age = df['Age'].mean()
print(f"Age column has {df['Age'].isna().sum()} / {df.shape[0]} missing values -- going to replace with average age {mean_age}")

# replace with average
missing = df['Age'].isna()
df.loc[missing, 'Age'] = mean_age

Age column has 177 / 891 missing values -- going to replace with average age 29.69911764705882


In [19]:
# pull out unique cabin types
def map_cabin_type(cabin_val):
    if pd.isna(cabin_val):
        return None
    else:
        # first char
        return cabin_val[0]

# convert cabin to cabin types
df['cabin_types'] = df['Cabin'].apply(map_cabin_type)

print('--- cabin_types frequencies ---')
print(df['cabin_types'].value_counts())
print('None rows:', df['cabin_types'].isna().sum())

--- cabin_types frequencies ---
cabin_types
C    59
B    47
D    33
E    32
A    15
F    13
G     4
T     1
Name: count, dtype: int64
None rows: 687


### Preprocess data to construct training matrices

What is not mentioned has been dropped
- `Survived`: keep as is (target variable)
- `SibSp`: z-normalize
- `Parch`: z-normalize
- `Fare`: z-normalize
- `Age`: would have liked to z-normalize, but 177/
- `Sex`: one-hot
- `cabin_types`: one-hot
- `Embarked`: one-hot
- `Pclass`: one-hot

In [41]:
# these columns need to be z-normalized
to_scale = ['SibSp', 'Parch', 'Fare', 'Age']
scaler = StandardScaler()
scaled_cols = scaler.fit(df[to_scale])

# these columns are categorical, need to be one-hot encoded
onehot = ['cabin_types', 'Embarked', 'Pclass', 'Sex', 'Pclass']
onehot_encoder = OneHotEncoder(handle_unknown='ignore')
onehot_cols = onehot_encoder.fit(df[onehot])

def vectorize_data(df, scaler, onehot_encoder):
    scaled_cols = scaler.transform(df[to_scale])
    
    onehot_cols = onehot_encoder.transform(df[onehot]).todense()
    n_onehot_cols = sum([len(cats) for cats in onehot_encoder.categories_])

    # create empty matrix
    examples = df.shape[0]
    n_cols = n_onehot_cols + len(to_scale)
    X = np.zeros((examples, n_cols))

    # fill out column-by-column
    i = 0
    X[: , i:i+len(to_scale)] = scaled_cols
    i += len(to_scale)
    X[: , i:i+n_onehot_cols] = onehot_cols
    print('X.shape:', X.shape)    

    return X

In [31]:
X = vectorize_data(df, scaler, onehot_encoder)

y_true = np.array(df['Survived'])
print('y.shape:', y_true.shape)

X.shape: (891, 25)
y.shape: (891,)


In [23]:
# split training data into train and validation split
X_train, X_val, y_train, y_val = train_test_split(X, y_true, test_size=0.2, random_state=417)

print('train size:', X_train.shape[0])
print('validation size:', X_val.shape[0])

train size: 712
validation size: 179


### Build model

In [24]:
import keras
from keras import layers

In [25]:
input_shape = X.shape[1]

# two-layer MLP
model = keras.Sequential([
    layers.Dense(input_shape, activation='relu'),
    layers.Dense(input_shape, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # prob. of survival
])
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [26]:
history = model.fit(
    X_train,
    y_train,
    epochs=20,
    batch_size=16,
    validation_data=(X_val, y_val)
)

Epoch 1/20


2024-04-17 21:35:58.104157: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


45/45 [==============================] - 0s 3ms/step - loss: 0.6208 - accuracy: 0.7135 - val_loss: 0.5225 - val_accuracy: 0.8436
Epoch 2/20
45/45 [==============================] - 0s 915us/step - loss: 0.5216 - accuracy: 0.7837 - val_loss: 0.4356 - val_accuracy: 0.8827
Epoch 3/20
45/45 [==============================] - 0s 817us/step - loss: 0.4743 - accuracy: 0.7978 - val_loss: 0.3928 - val_accuracy: 0.8827
Epoch 4/20
45/45 [==============================] - 0s 792us/step - loss: 0.4520 - accuracy: 0.8020 - val_loss: 0.3717 - val_accuracy: 0.8827
Epoch 5/20
45/45 [==============================] - 0s 830us/step - loss: 0.4434 - accuracy: 0.8062 - val_loss: 0.3644 - val_accuracy: 0.8827
Epoch 6/20
45/45 [==============================] - 0s 895us/step - loss: 0.4390 - accuracy: 0.8090 - val_loss: 0.3566 - val_accuracy: 0.8883
Epoch 7/20
45/45 [==============================] - 0s 865us/step - loss: 0.4326 - accuracy: 0.8118 - val_loss: 0.3501 - val_accuracy: 0.8939
Epoch 8/20
45/45 [=

### Evaluate

In [27]:
y_probs = model.predict(X_val, verbose=False)
THRESH = 0.5
y_pred = np.array(y_probs > THRESH, dtype=int).squeeze()

print('Accuracy:', '{:.3f}'.format(accuracy_score(y_val, y_pred)))
print('AUC:', '{:.3f}'.format(roc_auc_score(y_val, y_probs)))
print('\n', classification_report(y_val, y_pred))

Accuracy: 0.899
AUC: 0.903

               precision    recall  f1-score   support

           0       0.88      0.98      0.93       115
           1       0.96      0.75      0.84        64

    accuracy                           0.90       179
   macro avg       0.92      0.87      0.88       179
weighted avg       0.91      0.90      0.90       179



### Might as well make a submission to Kaggle :)

In [33]:
df_test = pd.read_csv('data/titanic_test.csv')

# impute age in the test data (with mean from training)
missing_test = df_test['Age'].isna()
df_test.loc[missing, 'Age'] = mean_age
# add cabin types column
df_test['cabin_types'] = df_test['Cabin'].apply(map_cabin_type)

# vectorize
X_test = vectorize_data(df_test, scaler, onehot_encoder)

X.shape: (418, 25)


In [34]:
submission_probs = model.predict(X_test)
submission_preds = np.array(submission_probs > THRESH, dtype=int).squeeze()

14/14 [==============================] - 0s 485us/step


In [40]:
df_submission = pd.DataFrame.from_dict({'PassengerId': df_test['PassengerId'], 'Survived': submission_preds})
df_submission.to_csv('submission.csv', index=False)